In [2]:
import os
import sys
import gzip
import json
from pathlib import Path
import csv

import pandas as pd
import s3fs
import pyarrow as pa
from pyarrow.json import read_json
import pyarrow.parquet as pq
import fastavro
import pygeohash
import snappy
import jsonschema
from jsonschema.exceptions import ValidationError
from jsonschema import validate


endpoint_url='https://storage.budsc.midwest-datascience.com'

current_dir = Path(os.getcwd()).absolute()
schema_dir = current_dir.joinpath('schemas')
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)


def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
        

    return records

In [3]:
records = read_jsonl_data()

print(json.dumps(records[0], sort_keys=False, indent=4))

{
    "airline": {
        "airline_id": 410,
        "name": "Aerocondor",
        "alias": "ANA All Nippon Airways",
        "iata": "2B",
        "icao": "ARD",
        "callsign": "AEROCONDOR",
        "country": "Portugal",
        "active": true
    },
    "src_airport": {
        "airport_id": 2965,
        "name": "Sochi International Airport",
        "city": "Sochi",
        "country": "Russia",
        "iata": "AER",
        "icao": "URSS",
        "latitude": 43.449902,
        "longitude": 39.9566,
        "altitude": 89,
        "timezone": 3.0,
        "dst": "N",
        "tz_id": "Europe/Moscow",
        "type": "airport",
        "source": "OurAirports"
    },
    "dst_airport": {
        "airport_id": 2990,
        "name": "Kazan International Airport",
        "city": "Kazan",
        "country": "Russia",
        "iata": "KZN",
        "icao": "UWKD",
        "latitude": 55.606201171875,
        "longitude": 49.278701782227,
        "altitude": 411,
        "timezone

In [4]:
# 3.1 a

In [5]:
def validate_jsonl_data(records):
    schema_path = schema_dir.joinpath('routes-schema.json')
    with open(schema_path) as f:
        schema = json.load(f)
#     with open('routes-schema.json') as f:
#         schema = json.load(f)
        
        validation_csv_path = results_dir.joinpath('results.csv')
        
    with open(validation_csv_path, 'w', encoding="utf-8") as f:
        fieldnames=['row_num', 'is_valid', 'msg']
        csv_writer = csv.DictWriter(f, fieldnames = fieldnames)
        csv_writer.writeheader()
        
        for i, record in enumerate(records):
            try:
                result=dict(
                row_num=i,
                is_valid=True,
                msg=record
                )
#                 validate(instance=records, schema = schema)
#                 with open('results.csv', mode='w') as csv_file:
#                     pass
            except ValidationError as e:
                result=dict(
                row_num=i,
                is_valid=False,
                msg=record
                )
#                 print(err)
#                 err = "Given JSON data is Invalid"
#                 return False, err
#             pass
            finally:
                csv_writer.writerow(result)
            

validate_jsonl_data(records)

In [6]:
pd.read_csv("results/results.csv")['is_valid'].value_counts()

True    67663
Name: is_valid, dtype: int64

In [7]:
# 3.1b

In [8]:
def create_avro_dataset(records):
    schema_path = schema_dir.joinpath('routes.avsc')
    data_path = results_dir.joinpath('routes.avro')
    
    with open(schema_path, 'r') as f:
        schema = json.load(f)
    
#     parsed_schema = fastavro.parse_schema(schema)
    
#     with open('routes.avro', 'wb') as out:
#         fastavro.writer(out, parsed_schema, records)

    print(json.dumps(schema, sort_keys=False, indent=4))
    
        
create_avro_dataset(records)

{
    "type": "record",
    "name": "Route",
    "namespace": "edu.bellevue.dsc650",
    "fields": [
        {
            "name": "airline",
            "type": {
                "type": "record",
                "name": "Airline",
                "fields": [
                    {
                        "name": "airline_id",
                        "type": "int",
                        "default": -1
                    },
                    {
                        "name": "name",
                        "type": "string",
                        "default": "NONE"
                    },
                    {
                        "name": "alias",
                        "type": "string",
                        "default": "NONE"
                    },
                    {
                        "name": "iata",
                        "type": "string",
                        "default": "NONE"
                    },
                    {
                        "name": "icao",
 

In [9]:
# 3.1 c

In [10]:
def create_parquet_dataset():
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    parquet_output_path = results_dir.joinpath('routes.parquet')
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            pass
            ## TODO: Use Apache Arrow to create Parquet table and save the dataset
        parq_output = read_json(parquet_output_path)
        pq.write_table(parq_output, 'routes.parquet')

create_parquet_dataset()

In [11]:
# 3.1 d

In [12]:
sys.path.insert(0, os.path.abspath('routes_pb2'))

import routes_pb2

def _airport_to_proto_obj(airport):
    obj = routes_pb2.Airport()
    if airport is None:
        return None
#     return obj
    if airport.get('airport_id') is None:
        return None
#     return obj
    
    obj.airport_id = airport.get('airport_id')
    if airport.get('name'):
        obj.name = airport.get('name')
    if airport.get('city'):
        obj.city = airport.get('city')
    if airport.get('iata'):
        obj.iata = airport.get('iata')
    if airport.get('icao'):
        obj.icao = airport.get('icao')
    if airport.get('altitude'):
        obj.altitude = airport.get('altitude')
    if airport.get('timezone'):
        obj.timezone = airport.get('timezone')
    if airport.get('dst'):
        obj.dst = airport.get('dst')
    if airport.get('tz_id'):
        obj.tz_id = airport.get('tz_id')
    if airport.get('type'):
        obj.type = airport.get('type')
    if airport.get('source'):
        obj.source = airport.get('source')

    obj.latitude = airport.get('latitude')
    obj.longitude = airport.get('longitude')

    return obj


def _airline_to_proto_obj(airline):
    obj = routes_pb2.Airline()
    ## TODO: Create an Airline obj using Protocol Buffers API
    if airline is None:
        return None
    if airline.get('airline_id') is None:
        return None
    obj.airline_id = airline.get('airline_id')
    if airline.get('name'):
        obj.name = airline.get('name')
    if airline.get('alias'):
        obj.alias = airline.get('alias')
    if airline.get('iata'):
        obj.iata = airline.get('iata')
    if airline.get('icao'):
        obj.icao = airline.get('icao')
    if airline.get('callsign'):
        obj.callsign = airline.get('callsign')
    if airline.get('country'):
        obj.country = airline.get('country')
    if airline.get('active'):
        obj.active = airline.get('active')
    else:
        obj.active = bool(None)
    
    return obj

def create_protobuf_dataset(records):
    routes = routes_pb2.Routes()
    for record in records:
        route = routes_pb2.Route()
        airline = _airline_to_proto_obj(record.get('airline', {}))
        if airline:
            route.airline.CopyFrom(airline)
        src_airport = _airport_to_proto_obj(record.get('src_airport', {}))
        if src_airport:
            route.src_airport.CopyFrom(src_airport)
        # TODO
        dst_airport = _airport_to_proto_obj(record.get('dst_airport', {}))
        if dst_airport:
            route.dst_airport.CopyFrom(dst_airport)
#         codeshare = record.get('codeshare')
        route.codeshare = record['codeshare']
        route.stops=1
        route.equipment[:] = record['equipment']

        routes.route.append(route)

    data_path = results_dir.joinpath('routes.pb')

    with open(data_path, 'wb') as f:
        f.write(routes.SerializeToString())
        
    compressed_path = results_dir.joinpath('routes.pb.snappy')
    
    with open(compressed_path, 'wb') as f:
        f.write(snappy.compress(routes.SerializeToString()))
        
create_protobuf_dataset(records)

In [13]:
# 3.1 e

In [14]:
comparison_csv_path = results_dir.joinpath('comparison.csv')

with open(comparison_csv_path, 'w', encoding="utf-8") as f:
    fieldnames = ['file', 'size_MB']
    
    csv_writer = csv.DictWriter(f, fieldnames=fieldnames)
    csv_writer.writeheader()
    
    for files in os.walk(results_dir):
        try:
            for file in files[2]:
                if file == "comparison.csv":
                    next
                else:
                    print(f"{file:>25}: {int(os.stat(results_dir.joinpath(file))[6]) / 1024 / 1024:.2f}MB")
                    
                    file_size = dict(file = file, size_MB = int(os.stat(results_dir.joinpath(file))[6]) / 1024 / 1024)
                    
                    csv_writer.writerow(file_size)
                    
        except FileNotFoundError:
            pass

              results.csv: 57.15MB
                routes.pb: 21.61MB
              routes.avro: 0.00MB
           routes.parquet: 0.00MB
         routes.pb.snappy: 3.59MB


In [15]:
# 3.2 a

In [16]:
def create_hash_dirs(records):
    geoindex_dir = results_dir.joinpath('geoindex')
    geoindex_dir.mkdir(exist_ok=True, parents=True)
    hashes = []
    for record in records:
        src_airport = record.get('src_airport', {})
        if src_airport:
            latitude = src_airport.get('latitude')
            longitude = src_airport.get('longitude')
            if latitude and longitude:
                ## TODO: use pygeohash.encode() to assign geohashes to the records and complete the hashes list
                hashes.append(pygeohash.encode(latitude, longitude, precision = 3))
    hashes.sort()
    three_letter = sorted(list(set([entry[:3] for entry in hashes])))
    hash_index = {value: [] for value in three_letter}
    for record in records:
        geohash = record.get('geohash')
        if geohash:
            hash_index[geohash[:3]].append(record)
    for key, values in hash_index.items():
        output_dir = geoindex_dir.joinpath(str(key[:1])).joinpath(str(key[:2]))
        output_dir.mkdir(exist_ok=True, parents=True)
        output_path = output_dir.joinpath('{}.jsonl.gz'.format(key))
        with gzip.open(output_path, 'w') as f:
            json_output = '\n'.join([json.dumps(value) for value in values])
            f.write(json_output.encode('utf-8'))
            
create_hash_dirs(records)

In [17]:
# 3.2 b

In [19]:
def airport_search(latitude, longitude):
    ## TODO: Create simple search to return nearest airport
    a = pygeohash.encode(latitude, longitude)
    dist = 0
    name = ''
    
    for i, record in enumerate(records):
        src_airport = record.get('src_airport', {})
        if src_airport:
            lat = src_airport.get('latitude')
            long = src_airport.get('longitude')
            airport_name = src_airport.get('name')
            if lat and long:
                a1 = pygeohash.encode(lat, long)
                    
                dist_n = pygeohash.geohash_approximate_distance(a, a1)
                if i==0:
                    dist = dist_n
                    name = airport_name
                        
                else:
                    if dist > dist_n:
                        dist = dist_n
                        name = airport_name
                            
        return(name)

    
airport_search(41.1499988, -95.91779)

'Sochi International Airport'

In [20]:
pygeohash.encode(41.1499988, -95.91779)

'9z7f174u17zb'